In [ ]:
import mne
import pandas as pd
from pathlib import Path
import re

def clear_all_ints(text):
    # \d matches any digit (0-9)
    return re.sub(r'\d+', '', text)

path = Path('/home/yhchoi/EEG_Data/Raw/EDF_20251222/')
patient_data = []  # List to store info for the DataFrame

for file in path.glob('*.edf'):
    try:
        raw = mne.io.read_raw_edf(file, preload=False)  # preload=False is faster if just reading info
        info = raw.info.get('subject_info', {})
        
        # 1. Safely extract name components (handles missing middle names)
        his_id = info.get('his_id', 'UnknownID')
        first = clear_all_ints(info.get('first_name', '').replace(',', ''))
        middle = clear_all_ints(info.get('middle_name', '').replace(',', ''))
        last = clear_all_ints(info.get('last_name', '').replace(',', ''))
        sex = info.get('sex', 'U')
        bday = info.get('birthday', None).strftime("%y%m%d_%H%M%S") if info.get('birthday') else 'UnknownBday'
        
        # 2. Build filename and timestamp
        meas_date = raw.info['meas_date']
        timestamp = meas_date.strftime("%y%m%d_%H%M%S") if meas_date else "000000_000000"
        
        name_part = f"{his_id}_{first}{middle}{last}"
        new_filename = f"{name_part}_{timestamp}.edf"
        new_file_path = file.with_name(new_filename)
        
        # 3. Rename the file
        file.rename(new_file_path)
        
        # 4. Store data for the DataFrame
        patient_data.append({
            'original_path': str(file),
            'new_path': str(new_file_path),
            'subject_id': his_id,
            'first_name': first,
            'last_name': last,
            'measurement_date': meas_date,
            'sex': sex,
            'birthday': bday
        })
        
        print(f"Renamed to: {new_file_path.name}")

    except Exception as e:
        print(f"Error processing {file.name}: {e}")

# 5. Create and save the DataFrame
df = pd.DataFrame(patient_data)
df.to_csv('patient_record.csv', index=False)
print("\nDataFrame created and saved to 'patient_record.csv'")